In [ ]:
pip install shap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler, Normalizer
from matplotlib.colors import ListedColormap
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, LeaveOneOut
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from itertools import combinations
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve
import seaborn as sns
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report
import shap

In [ ]:
names = [
    "KNN",
    "RF",
    "QDA",
    "SVM"
]

classifiers = [
    make_pipeline(StandardScaler(), KNeighborsClassifier()),
    make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42)),
    make_pipeline(StandardScaler(), QuadraticDiscriminantAnalysis()),
    make_pipeline(StandardScaler(), SVC(random_state=42, probability=True))
]


In [ ]:
# Lista de índices das bases de dados
indices = [17, 21, 23, 22, 24, 26, 27, 30, 39]

resultados_gerais = {name: {"accuracy": [], "precision": [], "recall": [], "f1_score": []} for name in names}

dfs = {}
for i in indices:
    df = pd.read_csv(f'/content/drive/MyDrive/data/finalVersion/base{i}_final.csv')
    df['number data'] = range(len(df))
    df = df.drop(columns=['id_reforma','sdv002','sdc002','mdv001_1','mdv001_2','mdv001_3','mdv001_4','sdu001_1','sdu001_2','data_hora_reforma','TipoAlarmeReleResistencia_id_tipo','TipoAlarmeResistencia_id_tipo','id_reacao'])
    df.rename(columns={'sdp002': 'pressure', 'sdt002': 'reactor temperature','sdt003':'pre-reactor temperature','sdu002_1':'humidity','sdu002_2':'outlet temperature','rotulo':'label'}, inplace=True)
    dfs[i] = df

combinacoes = [
    {"treinamento": [17, 21, 22, 24, 26, 27, 30, 39], "teste": [23]},
    {"treinamento": [17, 21, 23, 22, 26, 27, 30, 39], "teste": [24]},
    {"treinamento": [17, 21, 23, 22, 24, 27, 30, 39], "teste": [26]},
    {"treinamento": [23, 22, 24, 26, 27, 30, 39], "teste": [17, 21]},
    {"treinamento": [21, 23, 24, 26, 27, 30, 39], "teste": [17, 22]},
    {"treinamento": [21, 23, 22, 24, 26, 27, 39], "teste": [17, 30]},
    {"treinamento": [21, 23, 22, 24, 26, 27, 30], "teste": [17, 39]},
    {"treinamento": [17, 23, 22, 24, 26, 30, 39], "teste": [27, 21]},
    {"treinamento": [17, 21, 23, 24, 26, 30, 39], "teste": [27, 22]},
    {"treinamento": [17, 21, 23, 22, 24, 26, 39], "teste": [27, 30]},
    {"treinamento": [17, 21, 23, 22, 24, 26, 30], "teste": [27, 39]},
]

In [ ]:

for i, combinacao in enumerate(combinacoes):
    # Bases para treinamento
    dfs_treinamento = [dfs[idx] for idx in combinacao["treinamento"]]
    treino_df = pd.concat(dfs_treinamento, ignore_index=True)

    # Bases para teste
    dfs_teste = [dfs[idx] for idx in combinacao["teste"]]
    teste_df = pd.concat(dfs_teste, ignore_index=True)

    print(f"Combinação {i + 1}:")
    print(f"  Treinamento: Índices {combinacao['treinamento']} -> {len(treino_df)} linhas no total.")
    print(f"  Teste: Índices {combinacao['teste']} -> {len(teste_df)} linhas no total.")
    print('')

    # Separar atributos e rótulos
    X_train = treino_df.drop(columns=['label'])
    y_train = treino_df['label']
    X_test = teste_df.drop(columns=['label'])
    y_test = teste_df['label']

    for name, clf in zip(names, classifiers):
        # Validação cruzada
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cross_val_scores = cross_val_score(clf, X_train, y_train, cv=skf)

        print(f"Validação Cruzada - {name}:")
        print('Média dos Scores da Validação Cruzada:', np.mean(cross_val_scores))
        print('')

        # Teste final
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        conf_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, digits=4)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted')
        rec = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Armazenar resultados
        resultados_gerais[name]["accuracy"].append(acc)
        resultados_gerais[name]["precision"].append(prec)
        resultados_gerais[name]["recall"].append(rec)
        resultados_gerais[name]["f1_score"].append(f1)

        print(f"VALIDAÇÃO - Matriz de Confusão {name}:")
        print(conf_matrix)
        print(f"VALIDAÇÃO - Relatório de Classificação {name}:\n{class_report}")
        print('-' * 60)

        real = y_test.values  # Valores reais
        pred = y_pred         # Previsões do modelo

        # Criar o gráfico
        plt.figure(figsize=(12, 6))
        plt.plot(real, label='Real', linestyle='-', marker='o', alpha=0.7)
        plt.plot(pred, label='Predicted', linestyle='--', marker='x', alpha=0.7)
        #plt.title(f'Combinação {i+1} - {name}')
        plt.xlabel('Samples')
        plt.ylabel('Class')
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:
print("MÉDIAS GERAIS DAS MÉTRICAS PARA TODAS AS COMBINAÇÕES:")
for name in names:
    acc_mean = np.mean(resultados_gerais[name]["accuracy"])
    prec_mean = np.mean(resultados_gerais[name]["precision"])
    rec_mean = np.mean(resultados_gerais[name]["recall"])
    f1_mean = np.mean(resultados_gerais[name]["f1_score"])

    print(f"{name}:")
    print(f"  Média da Acurácia: {acc_mean:.4f}")
    print(f"  Média da Precisão: {prec_mean:.4f}")
    print(f"  Média do Recall: {rec_mean:.4f}")
    print(f"  Média do F1-Score: {f1_mean:.4f}")
    print('-' * 60)

In [ ]:
resultados_rf = resultados_gerais["RF"]

accuracy_values = resultados_rf['accuracy']
melhor_comb = np.argmax(accuracy_values)  # Índice da melhor combinação (maior accuracy)
pior_comb = np.argmin(accuracy_values)  # Índice da pior combinação (menor accuracy)

# Exibir os índices da melhor e pior combinação
print(f"Melhor Combinação (índice {melhor_comb + 1}): {accuracy_values[melhor_comb]:.4f}")
print(f"Pior Combinação (índice {pior_comb + 1}): {accuracy_values[pior_comb]:.4f}")

melhor_comb_treinamento = combinacoes[melhor_comb]["treinamento"]
melhor_comb_teste = combinacoes[melhor_comb]["teste"]

dfs_treinamento_melhor = [dfs[idx] for idx in melhor_comb_treinamento]
treino_df_melhor = pd.concat(dfs_treinamento_melhor, ignore_index=True)

dfs_teste_melhor = [dfs[idx] for idx in melhor_comb_teste]
teste_df_melhor = pd.concat(dfs_teste_melhor, ignore_index=True)

# Carregar as bases de dados para a pior combinação
pior_comb_treinamento = combinacoes[pior_comb]["treinamento"]
pior_comb_teste = combinacoes[pior_comb]["teste"]

dfs_treinamento_pior = [dfs[idx] for idx in pior_comb_treinamento]
treino_df_pior = pd.concat(dfs_treinamento_pior, ignore_index=True)

dfs_teste_pior = [dfs[idx] for idx in pior_comb_teste]
teste_df_pior = pd.concat(dfs_teste_pior, ignore_index=True)

# Treinamento e avaliação para a melhor combinação
X_train_melhor = treino_df_melhor.drop(columns=['label'])
y_train_melhor = treino_df_melhor['label']
X_train_melhor = np.array(X_train_melhor)
y_train_melhor = np.array(y_train_melhor)

X_test_melhor = teste_df_melhor.drop(columns=['label'])
y_test_melhor = teste_df_melhor['label']
X_test_melhor = np.array(X_test_melhor)
y_test_melhor = np.array(y_test_melhor)

# Treinamento e avaliação para a pior combinação
X_train_pior = treino_df_pior.drop(columns=['label'])
y_train_pior = treino_df_pior['label']
X_train_pior = np.array(X_train_pior)
y_train_pior = np.array(y_train_pior)

X_test_pior = teste_df_pior.drop(columns=['label'])
y_test_pior = teste_df_pior['label']
X_test_pior = np.array(X_test_pior)
y_test_pior = np.array(y_test_pior)


In [ ]:
print('SHAP - MELHOR COMBINAÇÃO:')
model = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
model.fit(X_train_melhor, y_train_melhor)
explainer = shap.Explainer(model.predict, X_train_melhor)

shap_values = explainer(X_test_melhor)

feature_names = treino_df_melhor.drop(columns=['label']).columns

shap.summary_plot(shap_values, X_test_melhor, feature_names=feature_names)


In [ ]:
feature_names = treino_df_melhor.drop(columns=['label']).columns
model = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
model.fit(X_train_melhor, y_train_melhor)

features = {"features": [0, 1, 2, 3, 4, 5], "kind": "both", "centered": True}
common_params = {
    "subsample": 100,
    "n_jobs": 2,
    "grid_resolution": 20,
    "random_state": 42,
}

# Ajustando o layout para 3 linhas e 2 colunas
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 12), sharey=True, constrained_layout=True)

# Criando os gráficos de PDP e ICE
display = PartialDependenceDisplay.from_estimator(
    model, X_test_melhor, **{**features, **common_params}, feature_names=feature_names, ax=axs, target=2
)
display.figure_.suptitle(f"ICE and PDP representations RF C11", fontsize=16)

# Removendo a legenda do eixo y dos subgráficos, exceto do primeiro
for ax in axs[:, 1:].flatten():  # Excluindo a primeira coluna
    ax.set_ylabel('')

plt.show()

In [ ]:
print('SHAP - PIOR COMBINAÇÃO:')
model = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
model.fit(X_train_pior, y_train_pior)
explainer = shap.Explainer(model.predict, X_train_pior)
shap_values = explainer(X_test_pior)

shap.summary_plot(shap_values, X_test_pior, feature_names=feature_names)

In [ ]:
model = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
model.fit(X_train_pior, y_train_pior)

features = {"features": [0,1,2, 3,4, 5], "kind": "both", "centered": True}
common_params = {
    "subsample": 100,
    "n_jobs": 2,
    "grid_resolution": 20,
    "random_state": 42,
    "percentiles": (0.01, 0.99)
}

# Ajustando o layout para 3 linhas e 2 colunas
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 12), sharey=True, constrained_layout=True)

# Criando os gráficos de PDP e ICE
display = PartialDependenceDisplay.from_estimator(
    model, X_test_pior, **{**features, **common_params}, feature_names=feature_names, ax=axs, target=2
)
display.figure_.suptitle(f"ICE and PDP representations RF C3", fontsize=16)

# Removendo a legenda do eixo y dos subgráficos, exceto do primeiro
for ax in axs[:, 1:].flatten():  # Excluindo a primeira coluna
    ax.set_ylabel('')

plt.show()


In [ ]:
def plot_shap_comparison(model, X_train, X_test, y_train, y_test, model_name="Model"):

    feature_names = treino_df_melhor.drop(columns=['label']).columns
    # Treinando o modelo
    model.fit(X_train, y_train)

    # Gerando previsões
    y_pred = model.predict(X_test)

    # Identificando previsões corretas e incorretas
    incorrect_true_labels = y_test[y_pred != y_test]
    incorrect_preds = X_test[y_pred != y_test]

    # Realizando a análise SHAP para previsões corretas e incorretas
    explainer = shap.Explainer(model.predict_proba, X_train)
    shap_values_incorrect = explainer(incorrect_preds)

    # Obtendo os nomes das classes
    print(f"SHAP Analysis for {name} (Incorrect Predictions)")
    shap.summary_plot(shap_values_incorrect, incorrect_preds, feature_names=feature_names, plot_type="bar")

    # Comparação entre previsões corretas e incorretas
    correct_preds = X_test[y_pred == y_test]
    correct_shap_values = explainer(correct_preds)

    class_names = clf.classes_  # Obter nomes das classes do classificador
    incorrect_shap_dfs = []
    correct_shap_dfs = []
    for class_index, class_name in enumerate(class_names):
        # Para previsões corretas
        incorrect_shap_dfs.append(pd.DataFrame(np.abs(shap_values_incorrect[:, :, class_index].values).mean(axis=0),
                                      columns=[f'SHAP Importance ({class_name})'],
                                      index=feature_names))

        # Para previsões incorretas
        correct_shap_dfs.append(pd.DataFrame(np.abs(correct_shap_values[:, :, class_index].values).mean(axis=0),
                                    columns=[f'SHAP Importance ({class_name})'],
                                    index=feature_names))


    incorrect_shap_df = pd.concat(incorrect_shap_dfs, axis=1)
    correct_shap_df = pd.concat(correct_shap_dfs, axis=1)

    # Combina as médias de SHAP para visualização comparativa
    comparison_df = pd.concat([correct_shap_df, incorrect_shap_df], axis=1, keys=["Correct", "Incorrect"])

    # Exibe o gráfico comparativo das importâncias SHAP
    comparison_df.plot(kind='bar', figsize=(10, 6))
    #plt.title(f"SHAP Importance Comparison for {name}")
    plt.ylabel("Average SHAP Importance")

    plt.show()

    # Exibe o dataframe comparativo (opcional)
    print("SHAP Importance Comparison (Correct vs Incorrect):")
    print(comparison_df)

model_rf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
# Analisando a Melhor Combinação
print("SHAP Analysis for Melhor Combinação (Random Forest)")
plot_shap_comparison(model_rf, X_train_melhor, X_test_melhor, y_train_melhor, y_test_melhor, "Melhor Combinação")

# Analisando a Pior Combinação
print("SHAP Analysis for Pior Combinação (Random Forest)")
plot_shap_comparison(model_rf, X_train_pior, X_test_pior, y_train_pior, y_test_pior, "Pior Combinação")